<a href="https://colab.research.google.com/github/PrakharPatni08/Gen-AI/blob/main/Conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional GAN:
A Conditional GAN is a special type of GAN where you give extra information to both the generator and the discriminator use to create output that match that condition

# Use Cases
1. Image to image translation
2. Label controlled
3. Text to image

#Workflow of Conditional GAN:
Step 1:- Data prepration:
1. Data load
2. Normalization
3. expand dimensions

Step 2:- Build Generator:
1. Inputs(noise vector, Labelled Inputs)
2. Label embeddinng
3. Concat Image+Label
4. Build Layers(conv2D, Leaky Relu, Dropout Layers,Flatten Layer, Dense Layer(single value output))

Step 3:- Build Discriminator
1. Inputs(Real and Fake image)
2. multiply nosie x label embeddings
3. Dense - Reshape
4. Unsampling(conv2D Transpose)
5. Activation (tanh)

Step 4:- Calculate loss and optimizers

Step 5:- Training Loop

Step 6:-Image Generation(20 image)

Step 7:- Generating using specific labels

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


In [ ]:


# -------------------------------
# Step 1: Data Preparation
# -------------------------------
def load_and_preprocess_data():
    # Example with MNIST dataset
    (x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

    # Normalize to [-1, 1]
    x_train = (x_train.astype("float32") - 127.5) / 127.5
    x_train = np.expand_dims(x_train, axis=-1)  # expand dims to (28,28,1)

    return x_train, y_train

x_train, y_train = load_and_preprocess_data()
BUFFER_SIZE = x_train.shape[0]
BATCH_SIZE = 64
LATENT_DIM = 100
NUM_CLASSES = 10  # MNIST digits


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# -------------------------------
# Step 2: Build Generator
# -------------------------------
def build_generator(latent_dim, num_classes):
    noise_input = layers.Input(shape=(latent_dim,))
    label_input = layers.Input(shape=(1,), dtype="int32")

    # Label embedding
    label_embedding = layers.Embedding(num_classes, latent_dim)(label_input)
    label_embedding = layers.Flatten()(label_embedding)

    # Combine noise + label
    combined_input = layers.multiply([noise_input, label_embedding])

    x = layers.Dense(7*7*256, use_bias=False)(combined_input)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((7, 7, 256))(x)

    x = layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    img_output = layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding="same", use_bias=False, activation="tanh")(x)

    return tf.keras.Model([noise_input, label_input], img_output, name="Generator")

generator = build_generator(LATENT_DIM, NUM_CLASSES)

In [ ]:
# -------------------------------
# Step 3: Build Discriminator
# -------------------------------
def build_discriminator(num_classes):
    img_input = layers.Input(shape=(28,28,1))
    label_input = layers.Input(shape=(1,), dtype="int32")

    # Label embedding
    label_embedding = layers.Embedding(num_classes, 28*28)(label_input)
    label_embedding = layers.Flatten()(label_embedding)
    label_embedding = layers.Reshape((28,28,1))(label_embedding)

    # Concatenate label with image
    x = layers.Concatenate()([img_input, label_embedding])

    x = layers.Conv2D(64, (5,5), strides=(2,2), padding="same")(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (5,5), strides=(2,2), padding="same")(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    return tf.keras.Model([img_input, label_input], out, name="Discriminator")

discriminator = build_discriminator(NUM_CLASSES)



In [ ]:
# -------------------------------
# Step 4: Losses & Optimizers
# -------------------------------
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:


# -------------------------------
# Step 5: Training Loop
# -------------------------------
@tf.function
def train_step(images, labels):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    random_labels = tf.random.uniform([BATCH_SIZE, 1], minval=0, maxval=NUM_CLASSES, dtype=tf.int32)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([noise, random_labels], training=True)

        real_output = discriminator([images, labels], training=True)
        fake_output = discriminator([generated_images, random_labels], training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, labels, epochs):
    for epoch in range(epochs):
        for i in range(0, dataset.shape[0], BATCH_SIZE):
            image_batch = dataset[i:i+BATCH_SIZE]
            label_batch = labels[i:i+BATCH_SIZE]

            if image_batch.shape[0] != BATCH_SIZE:  # Skip last incomplete batch
                continue

            g_loss, d_loss = train_step(image_batch, label_batch.reshape(-1,1))

        print(f"Epoch {epoch+1}/{epochs} | Gen Loss: {g_loss:.4f} | Disc Loss: {d_loss:.4f}")



In [ ]:


# -------------------------------
# Step 6: Image Generation
# -------------------------------
def generate_and_plot_images(generator, n_images=20, label=1):
    noise = tf.random.normal([n_images, LATENT_DIM])
    labels = tf.constant([[label]]*n_images)
    generated_images = generator([noise, labels], training=False)

    plt.figure(figsize=(10,10))
    for i in range(n_images):
        plt.subplot(4,5,i+1)
        plt.imshow((generated_images[i,:,:,0]+1)/2, cmap="gray")
        plt.axis("off")
    plt.show()



In [ ]:
# -------------------------------
# Run Training & Generate
# -------------------------------
EPOCHS = 50
train(x_train, y_train, EPOCHS)

Epoch 1/50 | Gen Loss: 0.8744 | Disc Loss: 1.2149
Epoch 2/50 | Gen Loss: 0.9351 | Disc Loss: 1.2192
Epoch 3/50 | Gen Loss: 1.2021 | Disc Loss: 1.2639
Epoch 4/50 | Gen Loss: 0.5802 | Disc Loss: 1.4692
Epoch 5/50 | Gen Loss: 0.7774 | Disc Loss: 1.1829
Epoch 6/50 | Gen Loss: 0.7234 | Disc Loss: 1.2715
Epoch 7/50 | Gen Loss: 0.7841 | Disc Loss: 1.1148
Epoch 8/50 | Gen Loss: 0.7357 | Disc Loss: 1.2346
Epoch 9/50 | Gen Loss: 0.6417 | Disc Loss: 1.2090
Epoch 10/50 | Gen Loss: 0.6503 | Disc Loss: 1.2330
Epoch 11/50 | Gen Loss: 0.6441 | Disc Loss: 1.4693
Epoch 12/50 | Gen Loss: 0.5374 | Disc Loss: 1.5461
Epoch 13/50 | Gen Loss: 0.7436 | Disc Loss: 1.3802


In [ ]:

# Generate 20 images of digit '7'
generate_and_plot_images(generator, n_images=20, label=7)
